# Installation des dépendances

In [ ]:
!sudo apt-get update && sudo apt-get -y install pkg-config libcairo2-dev pkg-config python3-dev

In [2]:
!pip install Pillow smartcrop matplotlib tensorflow numpy

In [3]:
%matplotlib inline

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
from pathlib import Path

import numpy as np
import tensorflow as tf
import tensorflow.keras as K

# Preparation du dataset

In [ ]:
from pathlib import Path

path_to_dataset = Path("data/dataset_v2")
path_to_cropped = path_to_dataset / "cropped"
path_to_models = Path("out/models")

img_height = 300
img_width = 300

In [ ]:
import tensorflow as tf
from modules import dataload

training_dataset = dataload.create_set(path_to_cropped, "training")
validation_dataset = dataload.create_set(path_to_cropped, "validation")

class_names = training_dataset.class_names
print(f"{class_names = }")
print(f"{len(class_names) = }")

training_dataset = training_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

#TODO : Ajouter un split de test pour comparer les differents types de modèles

In [ ]:
import numpy as np
from tensorflow import keras as K

image_batch, _ = next(iter(training_dataset))
first_image = image_batch[0]
print(f"Initial image values interval : [{np.min(first_image)}, {np.max(first_image)}]")

normalization_layer = K.layers.Rescaling(1./255)
normalized_ds = training_dataset.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]

# Notice the pixel values are now in `[0,1]`.
print(f"Normalized image values interval : [{np.min(first_image)}, {np.max(first_image)}]")

# Modèle simple

In [ ]:
from modules.model import create_model

num_classes = len(class_names)

model = create_model(
    input_shape=(img_height, img_width, 3),
    output_dim=num_classes
)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

## Entrainement du modèle simple

In [ ]:
epochs=10
history = model.fit(
  training_dataset,
  validation_data=validation_dataset,
  epochs=epochs
)

## Visualisation des métriques

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Sauvegarde

In [ ]:
from modules.model import save_model
save_model(
    model_folder=path_to_models / "simple",
    model=model,
)

# Augmentation du modele

## Illustration de la couche d'augmentation

In [ ]:
data_augmentation = K.Sequential([
    K.layers.RandomFlip(
        "horizontal",
        input_shape=(img_height, img_width, 3)
    ),
    K.layers.RandomRotation(0.1),
    K.layers.RandomZoom(0.1),
])

plt.figure(figsize=(10, 10))
for images, _ in training_dataset.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

## Déclaration du modèle augmenté

In [ ]:
augmented_model = create_model(
    input_shape=(img_height, img_width, 3),
    output_dim=num_classes,
    augmented=True,
)

In [ ]:
augmented_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [ ]:
augmented_model.summary()

## Entrainement du modèle augmenté

In [ ]:
history_aug = augmented_model.fit(
    training_dataset,
    validation_data=validation_dataset,
    epochs=epochs
)

## Sauvegarde et affichage des métriques

In [ ]:
acc = history_aug.history['accuracy']
val_acc = history_aug.history['val_accuracy']

loss = history_aug.history['loss']
val_loss = history_aug.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Sauvegarde

In [ ]:
from modules.model import save_model
save_model(
    model_folder=path_to_models / "augmented",
    model=augmented_model,
)

# Résultats

## Exemple d'inférence

In [ ]:
import os
from PIL import Image

test_file_url = "data/dataset/other/albizia_coriaria/0b6917a968f1dd06d388133971761f0d.jpg"
im = Image.open(os.path.abspath("./" + test_file_url)) 

sunflower_path = tf.keras.utils.get_file(fname='test-file', origin='file://'+os.path.abspath("./" + test_file_url))

img = tf.keras.utils.load_img(
    sunflower_path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

display(im)

## Comparaison du modèle simple et du modèle augmenté

In [ ]:
## TODO maybe ?